<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/04_dynamics/01_pfuzzy_discrete_predator_prey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema p-Fuzzy Discreto: Predador-Presa

## 📘 Introdução

Este notebook implementa um **sistema dinâmico p-fuzzy discreto** para modelar a interação entre **presas e predadores**, baseado no exemplo do livro:

> Barros, L. C., Bassanezi, R. C., & Lodwick, W. A. (2017).  
> *A First Course in Fuzzy Logic, Fuzzy Dynamical Systems, and Biomathematics*

---

## 🎯 O que é um Sistema p-Fuzzy?

Um **sistema p-fuzzy** é um sistema dinâmico onde a função de evolução é definida por **regras fuzzy** ao invés de equações matemáticas explícitas.

### Sistema Discreto:
$$x_{n+1} = x_n + f(x_n)$$

Onde $f(x_n)$ é determinado por um **Sistema de Inferência Fuzzy (FIS)**.

---

## 🦊 Modelo Predador-Presa

O modelo clássico de Lotka-Volterra descreve a interação entre duas espécies:
- **Presas** (população x₁)
- **Predadores** (população x₂)

### Regras Fuzzy do Livro (Figura 4.15):

A base de regras considera **4 níveis linguísticos** para cada espécie:
- **B** (Baixa)
- **MB** (Média-Baixa)  
- **MA** (Média-Alta)
- **A** (Alta)

E define **16 regras** que determinam a variação populacional.

---

## 🚀 Vamos começar!

## Instalação e Imports

In [ ]:
# Instalar pyfuzzy-toolbox
!pip install pyfuzzy-toolbox matplotlib numpy -q

In [ ]:
import fuzzy_systems as fs
import numpy as np
import matplotlib.pyplot as plt

print("✅ Bibliotecas importadas com sucesso!")

---

## 1️⃣ Criar o Sistema de Inferência Fuzzy

Vamos criar um FIS que implementa as **16 regras fuzzy** do livro.

In [ ]:
# Criar sistema Mamdani para predador-presa
fis = fs.MamdaniSystem(name="Predador-Presa Discreto")

# ========================================
# ENTRADAS: Presas (x) e Predadores (y)
# ========================================

# Universo de discurso: [0, 100] para ambas
fis.add_input('prey', (0, 100))
fis.add_input('predator', (0, 100))

# Termos linguísticos (4 níveis como no livro)
for var in ['prey', 'predator']:
    fis.add_term(var, 'B', 'gaussian', (12.5, 8))    # Baixa
    fis.add_term(var, 'MB', 'gaussian', (37.5, 8))   # Média-Baixa
    fis.add_term(var, 'MA', 'gaussian', (62.5, 8))   # Média-Alta
    fis.add_term(var, 'A', 'gaussian', (87.5, 8))    # Alta

print("✅ Variáveis de entrada criadas:")
print(f"   • prey: 4 termos (B, MB, MA, A)")
print(f"   • predator: 4 termos (B, MB, MA, A)")

In [ ]:
# ========================================
# SAÍDAS: Variação de Presas e Predadores
# ========================================

# Saída 1: Δprey (variação de presas)
fis.add_output('d_prey', (-30, 30))
fis.add_term('d_prey', 'declinio_forte', 'triangular', (-30, -30, -15))
fis.add_term('d_prey', 'declinio', 'triangular', (-20, -10, 0))
fis.add_term('d_prey', 'estavel', 'triangular', (-5, 0, 5))
fis.add_term('d_prey', 'crescimento', 'triangular', (0, 10, 20))
fis.add_term('d_prey', 'crescimento_forte', 'triangular', (15, 30, 30))

# Saída 2: Δpredator (variação de predadores)
fis.add_output('d_predator', (-30, 30))
fis.add_term('d_predator', 'declinio_forte', 'triangular', (-30, -30, -15))
fis.add_term('d_predator', 'declinio', 'triangular', (-20, -10, 0))
fis.add_term('d_predator', 'estavel', 'triangular', (-5, 0, 5))
fis.add_term('d_predator', 'crescimento', 'triangular', (0, 10, 20))
fis.add_term('d_predator', 'crescimento_forte', 'triangular', (15, 30, 30))

print("\n✅ Variáveis de saída criadas:")
print(f"   • prey (Δx): 5 termos")
print(f"   • predator (Δy): 5 termos")

### Base de Regras (16 regras do livro)

As regras implementam a lógica:
- **Presas:**
  - Crescem quando há poucos predadores
  - Decrescem quando há muitos predadores
  
- **Predadores:**
  - Crescem quando há muitas presas (alimento)
  - Decrescem quando há poucas presas

In [ ]:
# ========================================
# BASE DE REGRAS (16 regras)
# ========================================

# Definir regras usando sintaxe de tupla plana
# Formato: ('prey_term', 'predator_term', 'd_prey_term', 'd_predator_term')

rules = [
    # Linha 1: prey = B (baixa)
    ('B', 'B', 'crescimento', 'declinio'),
    ('B', 'MB', 'declinio', 'declinio'),
    ('B', 'MA', 'declinio_forte', 'declinio_forte'),
    ('B', 'A', 'declinio_forte', 'declinio_forte'),
    
    # Linha 2: prey = MB (média-baixa)
    ('MB', 'B', 'crescimento', 'crescimento'),
    ('MB', 'MB', 'estavel', 'estavel'),
    ('MB', 'MA', 'declinio', 'declinio'),
    ('MB', 'A', 'declinio_forte', 'declinio_forte'),
    
    # Linha 3: prey = MA (média-alta)
    ('MA', 'B', 'crescimento_forte', 'crescimento'),
    ('MA', 'MB', 'crescimento', 'crescimento'),
    ('MA', 'MA', 'estavel', 'estavel'),
    ('MA', 'A', 'declinio', 'estavel'),
    
    # Linha 4: prey = A (alta)
    ('A', 'B', 'crescimento_forte', 'crescimento_forte'),
    ('A', 'MB', 'crescimento', 'crescimento_forte'),
    ('A', 'MA', 'estavel', 'crescimento'),
    ('A', 'A', 'estavel', 'estavel')
]

# Adicionar regras ao FIS usando a nova sintaxe
fis.add_rules(rules)

print(f"\n✅ Base de regras criada: {len(fis.rule_base.rules)} regras")
print("\n📋 Exemplo de regras:")
print("   1. SE prey=B E predator=B ENTÃO Δprey=crescimento, Δpredator=declinio")
print("   2. SE prey=MB E predator=MB ENTÃO Δprey=estavel, Δpredator=estavel")
print("   3. SE prey=A E predator=B ENTÃO Δprey=crescimento_forte, Δpredator=crescimento_forte")

---

## 2️⃣ Criar o Sistema p-Fuzzy Discreto

Agora vamos criar o sistema dinâmico discreto usando `PFuzzyDiscrete`.

In [ ]:
# Criar sistema p-fuzzy discreto
pfuzzy = fs.dynamics.PFuzzyDiscrete(
    fis=fis,
    mode='absolute',  # x_{n+1} = x_n + f(x_n)
    state_vars=['prey', 'predator'],
    dt=1.0  # Passo de tempo (1 unidade)
)

print("✅ Sistema p-Fuzzy Discreto criado!")
print(f"\n📊 Configuração:")
print(f"   • Modo: absolute (x_n+1 = x_n + Δx)")
print(f"   • Variáveis de estado: {pfuzzy.state_vars}")
print(f"   • Passo de tempo: dt = {pfuzzy.dt}")

---

## 3️⃣ Simulação do Sistema

Vamos simular o sistema com diferentes condições iniciais.

In [ ]:
# Condições iniciais (mesmas do livro)
x0 = {'prey': 50, 'predator': 40}

# Número de passos de tempo
n_steps = 100

print(f"🚀 Iniciando simulação...")
print(f"\n📊 Condições iniciais:")
print(f"   • Presas (x₁): {x0['prey']}")
print(f"   • Predadores (x₂): {x0['predator']}")
print(f"\n⏱️  Simulando {n_steps} passos de tempo...")

# Simular
trajectory = pfuzzy.simulate(x0=x0, n_steps=n_steps)

print(f"\n✅ Simulação concluída!")
print(f"\n📈 Estatísticas finais:")
print(f"   Presas:")
print(f"      • Inicial: {trajectory[0, 0]:.2f}")
print(f"      • Final: {trajectory[-1, 0]:.2f}")
print(f"      • Mínimo: {trajectory[:, 0].min():.2f}")
print(f"      • Máximo: {trajectory[:, 0].max():.2f}")
print(f"\n   Predadores:")
print(f"      • Inicial: {trajectory[0, 1]:.2f}")
print(f"      • Final: {trajectory[-1, 1]:.2f}")
print(f"      • Mínimo: {trajectory[:, 1].min():.2f}")
print(f"      • Máximo: {trajectory[:, 1].max():.2f}")

---

## 4️⃣ Visualizações

### 📊 Dinâmica Temporal

In [ ]:
# Plot da dinâmica temporal
fig, ax = pfuzzy.plot_trajectory(
    variables=['prey', 'predator'],
    figsize=(12, 6),
    title='Sistema Predador-Presa Discreto',
    xlabel='Tempo (passos)',
    ylabel='População'
)

plt.show()

### 🔄 Espaço de Fase

O **espaço de fase** mostra a trajetória no plano (presas × predadores).

In [ ]:
# Plot do espaço de fase
fig, ax = pfuzzy.plot_phase_space(
    'prey', 'predator',
    figsize=(10, 10),
    title='Espaço de Fase: Predador-Presa (Discreto)'
)

plt.show()

---

## 5️⃣ Múltiplas Condições Iniciais

Vamos testar diferentes populações iniciais para observar o comportamento do sistema.

In [ ]:
# Diferentes condições iniciais
initial_conditions = [
    {'prey': 30, 'predator': 20},
    {'prey': 50, 'predator': 50},
    {'prey': 70, 'predator': 30},
    {'prey': 40, 'predator': 60}
]

colors = ['blue', 'red', 'green', 'purple']

# Criar figura
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

print("🔄 Simulando múltiplas condições iniciais...\n")

for i, x0 in enumerate(initial_conditions):
    print(f"{i+1}. Presas={x0['prey']}, Predadores={x0['predator']}")
    
    # Simular
    traj = pfuzzy.simulate(x0=x0, n_steps=100)
    time = pfuzzy.time
    
    # Plot temporal
    ax1.plot(time, traj[:, 0], '--', color=colors[i], alpha=0.5, linewidth=1.5)
    ax1.plot(time, traj[:, 1], '-', color=colors[i], linewidth=2, 
             label=f"IC{i+1}: ({x0['prey']}, {x0['predator']})")
    
    # Plot espaço de fase
    ax2.plot(traj[:, 0], traj[:, 1], color=colors[i], linewidth=2, alpha=0.7)
    ax2.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=10)
    ax2.plot(traj[-1, 0], traj[-1, 1], 's', color=colors[i], markersize=8)

# Configurar plot temporal
ax1.set_xlabel('Tempo (passos)', fontsize=12)
ax1.set_ylabel('População', fontsize=12)
ax1.set_title('Dinâmica Temporal (linha sólida=predador, tracejada=presa)', 
              fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Configurar plot espaço de fase
ax2.set_xlabel('Presas', fontsize=12)
ax2.set_ylabel('Predadores', fontsize=12)
ax2.set_title('Espaço de Fase (o=inicial, □=final)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Simulações concluídas!")

---

## 6️⃣ Exportar Resultados

Podemos exportar a trajetória para um arquivo CSV.

In [ ]:
# Exportar para CSV
pfuzzy.to_csv('/tmp/predator_prey_discrete.csv')

print("✅ Dados exportados para: /tmp/predator_prey_discrete.csv")
print("\n📄 Primeiras 5 linhas:")

# Ler e mostrar primeiras linhas
import pandas as pd
df = pd.read_csv('/tmp/predator_prey_discrete.csv')
print(df.head())

---

## 🎯 Conclusões

### ✅ O que aprendemos:

1. **Sistema p-Fuzzy Discreto:**
   - Evolução por passos discretos: $x_{n+1} = x_n + \Delta x$
   - A variação $\Delta x$ é determinada por regras fuzzy

2. **Dinâmica Predador-Presa:**
   - Comportamento oscilatório (ciclos)
   - Presas e predadores em equilíbrio dinâmico
   - Diferentes condições iniciais convergem para padrões similares

3. **Vantagens da Abordagem Fuzzy:**
   - **Interpretabilidade:** Regras linguísticas claras
   - **Flexibilidade:** Fácil ajustar comportamentos
   - **Conhecimento especialista:** Não precisa de parâmetros matemáticos precisos

---

## 🧪 Experimentos Sugeridos:

1. **Modifique as regras fuzzy:**
   - O que acontece se predadores crescem mais rápido?
   - E se as presas tiverem maior resiliência?

2. **Altere as funções de pertinência:**
   - Use funções triangulares ao invés de gaussianas
   - Varie os valores de centro e dispersão

3. **Teste diferentes condições iniciais:**
   - O que acontece com populações muito desequilibradas?
   - Existe um ponto de equilíbrio?

4. **Compare com o modelo contínuo:**
   - Execute o notebook `02_pfuzzy_continuous_predator_prey.ipynb`
   - Observe as diferenças entre os dois modelos

---

## 📚 Referências:

- Barros, L. C., Bassanezi, R. C., & Lodwick, W. A. (2017). *A First Course in Fuzzy Logic, Fuzzy Dynamical Systems, and Biomathematics*
- Documentação: https://github.com/1moi6/pyfuzzy-toolbox

---

**🤖 Notebook criado com pyfuzzy-toolbox**